In [1]:
import sys
import os
import numpy as np

from codingTools import *


In [2]:
######LOAD RAW IMAGE######
#dtype --> > stands for big endian, i stands for integer, 2 stands for 2 bytes
originalFile="../images/03508649.1_512_512_2_0_12_0_0_0d.raw" #int16
#originalFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0.raw" #uint8
ImageRaw = load_image_raw(originalFile, 1, 512, 512, "int16") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
#ImageRaw = load_image_raw(originalFile, 1, 2560, 2048, "uint8") #https://numpy.org/doc/stable/reference/arrays.dtypes.html
original_image = ImageRaw.load_image_raw()
##############################

In [3]:
########STATISTICS##########
statistics_data = statistics(original_image)
entropy = statistics_data.compute_entropy()
print("order zero entropy of the image: ",entropy)

order zero entropy of the image:  10.431490855522489


In [4]:
######SAVE RAW IMAGE######
# decodedFile="../images/n1_GRAY.1_2560_2048_1_0_8_0_0_0_copy.raw"
# ImageRawDecoded = save_image_raw(decodedFile)
# ImageRawDecoded.save_image_raw(original_image,"uint8")
###########

In [5]:
### Quantize ###
quantize = Quantizer(1, 10)
image_data_quantized = ImageRaw.get_image_structure_empty()
quantize.quantize(original_image, image_data_quantized)
quant_statics = statistics(image_data_quantized).compute_entropy()
print(f"Entropia: {quant_statics}")


Check Quantize: [[[ 2560.  2636. -3225. ... -2483. -2918. -3148.]
  [ 2432.  2713.  2739. ... -2431. -2739. -2918.]
  [ 2534.  2739. -3020. ... -2380. -2918. -2508.]
  ...
  [ 2688.  3046. -2508. ... -1945. -2431. -2508.]
  [ 1920.  2176.  3020. ... -1791.  3174. -2713.]
  [ 1945.  2252.  2508. ... -2713. -2918.  2867.]]]
Entropia: 8.427372356811585


In [6]:
### Dequantize ###
dequantize = Quantizer(1, 10)
image_data_dequantized = ImageRaw.get_image_structure_empty()
dequantize.dequantize(image_data_quantized, image_data_dequantized)

In [7]:
### PAE ###
pae = Metrics(original_image, image_data_dequantized)
print(f"Pae: {pae.calculate_pae()}")
print(pae.PAE_data)


Pae: 9.0
9.0


In [8]:
### MSE ###
mse = Metrics(original_image, image_data_dequantized)
print(f"Mse: {mse.calculate_mse()}")

Mse: 28.720321655273438


In [9]:
### PSNR ###
psnr = Metrics(original_image, image_data_dequantized)
print(f"Psnr: {psnr.calculate_psnr()} dB")

Psnr: 75.72710570410173 dB


In [10]:
###BITPACKING CODING###
# bitpacking_coding = BitPacking("bitpacket.txt" ,original_image)
# bitpacking_coding.coding()
# original_size_in_bytes = ImageRaw.get_original_size_in_bytes()
# CR = original_size_in_bytes / bitpacking_coding.size_of_coded_file
# print(original_size_in_bytes)
# print(bitpacking_coding.size_of_coded_file)

In [11]:
### S-transform forward ###
array = [1,2,3,4,8,6,10,8]
transform = Wavelet(array, 1)
array_transform = transform.s_tranform_forward(array)
print(f"Array modificado: {array_transform}")


Array modificado: [ 2  4  7  9 -1 -1  2  2]


In [12]:
### S-transform Inverse ###
array_inv = transform.s_transform_inverse(array_transform)
print(f"Array invertido: {array_inv}")


Array invertido: [ 1  2  3  4  8  6 10  8]


In [13]:
print(original_image)

[[[ 25601  26369 -32255 ... -24831 -29183 -31487]
  [ 24321  27137  27393 ... -24319 -27391 -29183]
  [ 25345  27393 -30207 ... -23807 -29183 -25087]
  ...
  [ 26881  30465 -25087 ... -19455 -24319 -25087]
  [ 19201  21761  30209 ... -17919  31745 -27135]
  [ 19457  22529  25089 ... -27135 -29183  28673]]]


In [17]:
### S-transform Forward image ###
transform_image = Wavelet(original_image, 2)
#print(len(original_image[0][0]))
transform = transform_image.handle_transform_forward()
print(len(transform[0]))
print(transform)


512
[[ 31489 -16399  -2830 ...   2944   4608   2048]
 [-22319  -3358   9954 ...   6400   3840    512]
 [-12670   7826  24530 ...   -256   2816  -3072]
 ...
 [ 13056  -1536   5632 ...   -256  17664   3584]
 [  2432   1280  -2048 ... -13568  -4352   6400]
 [  -512   2816  -3584 ...   7936  -2048 -14336]]
